In [9]:
%matplotlib inline
import sys
import random
import math
import numpy as np
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.providers.aer.noise import NoiseModel
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
backend = Aer.get_backend('qasm_simulator')

coupling_map = None
noise_model = None
basis_gates = None

Credentials are already in use. The existing account in the session will be replaced.


In [40]:
def loss(counts, shots):
    return error(counts, shots) ** 2
    
def error(counts, shots):
    count_00 = counts.get("00", 0)
    count_01 = counts.get("01", 0)
    count_10 = counts.get("10", 0)
    count_11 = counts.get("11", 0)
    return (abs(count_00 - count_11) + count_10 + count_01) / shots

def verify_loss(counts, shots):
    return verify_error(counts, shots) ** 2
    
def verify_error(counts, shots):
    count_0 = counts.get("0", 0)
    count_1 = counts.get("1", 0)    
    return (shots - count_0) / shots + (count_1 / shots)    

def gradient(theta_x1, theta_y, theta_x2, lr, shots):
    # TODO: Look into Jacobian matrices; for now generate the gradient by sampling
    gradient_theta_x1 = (error(run(theta_x1 + lr, 0, 0, shots), shots) - error(run(theta_x1 - lr, 0, 0, shots), shots)) / (2 * lr)
    gradient_theta_x1 += (verify_error(run(theta_x1 + lr, 0, 0, shots, func=get_verify_circ), shots) - verify_error(run(theta_x1 - lr, 0, 0, shots, func=get_verify_circ), shots)) / (2 * lr)

    gradient_theta_y = (error(run(0, theta_y + lr, 0, shots), shots) - error(run(0, theta_y - lr, 0, shots), shots)) / (2 * lr)
    gradient_theta_y += (verify_error(run(0, theta_y + lr, 0, shots, func=get_verify_circ), shots) - verify_error(run(0, theta_y - lr, 0, shots, func=get_verify_circ), shots)) / (2 * lr)

    gradient_theta_x2 = (error(run(0, 0, theta_x2 + lr, shots), shots) - error(run(0, 0, theta_x2 - lr, shots), shots)) / (2 * lr)
    gradient_theta_x2 += (verify_error(run(0, 0, theta_x2 + lr, shots, func=get_verify_circ), shots) - verify_error(run(0, 0, theta_x2 - lr, shots, func=get_verify_circ), shots)) / (2 * lr)
    
    return (gradient_theta_x1, gradient_theta_y, gradient_theta_x2)

def get_circ(theta_x1, theta_y, theta_x2):
    circ = QuantumCircuit(2, 2)

    circ.rx(theta_x1, 0)
    circ.ry(theta_y, 0)
    circ.rx(theta_x2, 0)
    circ.cx(0, 1)

    circ.measure(range(2), range(2))
    return circ

def get_verify_circ(theta_x1, theta_y, theta_x2):
    circ = QuantumCircuit(1, 1)

    circ.rx(theta_x1, 0)
    circ.ry(theta_y, 0)
    circ.rx(theta_x2, 0)
    # measure in the -x basis to make sure we won't end with |00>-|11>
    circ.ry(-math.pi/2, 0);

    circ.measure(range(1), range(1))
    return circ

def run(theta_x1, theta_y, theta_x2, shots, func=get_circ, error=error, disp=False):
    circ = func(theta_x1, theta_y, theta_x2)
    job = execute(circ, backend, shots=shots, coupling_map=coupling_map, noise_model=noise_model, basis_gates=basis_gates)
    results = job.result()
    counts = results.get_counts()
    if disp:
        print("%s error: %.4f [%.10f %.10f %.10f]" % (counts, error(counts, shots), theta_x1, theta_y, theta_x2))
    return counts

In [45]:
from IPython.core.debugger import set_trace
def prepare_bell_state_circuit(shots, lr=0.001, random_parameter_init=False, max_iterations=1000, noisy=False):    
    if noisy:    
        # Generate an Aer noise model for device
        device = provider.get_backend('ibmq_16_melbourne')
        properties = device.properties()    
        coupling_map = device.configuration().coupling_map
        noise_model = NoiseModel.from_backend(properties)
        basis_gates = noise_model.basis_gates
    else:
        coupling_map = None
        noise_model = None
        basis_gates = None

    if random_parameter_init:
        theta_x1 = random.uniform(-math.pi, math.pi)
        theta_y = random.uniform(-math.pi / 2.0, math.pi / 2.0)
        theta_x2 = random.uniform(-math.pi, math.pi)
    else:
        theta_x1 = 0
        theta_y = 0
        theta_x2 = 0

    current_loss = sys.maxsize

    for i in range(max_iterations):
        reg_counts = run(theta_x1, theta_y, theta_x2, shots, disp=False)
        reg_loss = loss(reg_counts, shots)
        ver_counts = run(theta_x1, theta_y, theta_x2, shots, func=get_verify_circ, error=verify_error, disp=False)
        ver_loss = verify_loss(ver_counts, shots)
        iteration_loss = reg_loss + ver_loss
        if (iteration_loss < current_loss):
            print("Iteration %4d => Loss %.4f (%.4f %.4f)" % (i, iteration_loss, reg_loss, ver_loss))
            print("Counts: %s %s" % (reg_counts, ver_counts))
#             print("Current thetas: [%.10f %.10f]" % (theta_x, theta_y))
            current_loss = iteration_loss

        if (iteration_loss > lr):
            gradient_theta_x1, gradient_theta_y, gradient_theta_x2 = gradient(theta_x1, theta_y, theta_x2, lr, shots)
#             print("g[%.10f %.10f %.10f]" % (gradient_theta_x1, gradient_theta_y, gradient_theta_x2))
            theta_x1 -= gradient_theta_x1 * lr
            theta_x1 = max(-math.pi, min(theta_x1, math.pi))
            #theta_x1 %= np.pi
            theta_y -= gradient_theta_y * lr
            #theta_y %= np.pi/2
            theta_y = max(-math.pi / 2.0, min(theta_y, math.pi / 2.0))
            theta_x2 -= gradient_theta_x2 * lr
            #theta_x2 %= np.pi
            theta_x2 = max(-math.pi, min(theta_x2, math.pi))
#             print("t[%.10f %.10f %.10f]" % (theta_x1, theta_y, theta_x2))            
        else:
            break

    #set_trace()
    print("\nNumber of iterations: %d" % i)
    print("%.10f %.10f %.10f (error: %.10f verify_error: %.10f)" % (theta_x1, theta_y, theta_x2, 
                                                error(run(theta_x1, theta_y, theta_x2, shots, disp=True), shots),
                                                verify_error(run(theta_x1, theta_y, theta_x2, shots, func=get_verify_circ, error=verify_error, disp=True), shots)))

    circ = get_circ(theta_x1, theta_y, theta_x2)
    circ.draw()

In [50]:
prepare_bell_state_circuit(1) # this will never converge with only 1 shot

Iteration    0 => Loss 1.0000 (1.0000 0.0000)
Counts: {'00': 1} {'0': 1}

Number of iterations: 999
{'11': 1} error: 1.0000 [0.8584073464 1.5707963268 3.1415926536]
{'0': 1} error: 0.0000 [0.8584073464 1.5707963268 3.1415926536]
0.8584073464 1.5707963268 3.1415926536 (error: 1.0000000000 verify_error: 0.0000000000)


In [47]:
prepare_bell_state_circuit(10)

Iteration    0 => Loss 2.9600 (1.0000 1.9600)
Counts: {'00': 10} {'0': 3, '1': 7}
Iteration    1 => Loss 1.6400 (1.0000 0.6400)
Counts: {'00': 10} {'0': 6, '1': 4}
Iteration    5 => Loss 1.3600 (0.3600 1.0000)
Counts: {'00': 8, '11': 2} {'0': 5, '1': 5}
Iteration   33 => Loss 0.3600 (0.0000 0.3600)
Counts: {'00': 5, '11': 5} {'0': 7, '1': 3}
Iteration   72 => Loss 0.3200 (0.1600 0.1600)
Counts: {'00': 3, '11': 7} {'0': 8, '1': 2}
Iteration   73 => Loss 0.2000 (0.0400 0.1600)
Counts: {'00': 4, '11': 6} {'0': 8, '1': 2}
Iteration   89 => Loss 0.1600 (0.1600 0.0000)
Counts: {'00': 3, '11': 7} {'0': 10}
Iteration   94 => Loss 0.0800 (0.0400 0.0400)
Counts: {'00': 4, '11': 6} {'0': 9, '1': 1}
Iteration  188 => Loss 0.0400 (0.0400 0.0000)
Counts: {'00': 6, '11': 4} {'0': 10}
Iteration  278 => Loss 0.0000 (0.0000 0.0000)
Counts: {'00': 5, '11': 5} {'0': 10}

Number of iterations: 278
{'00': 8, '11': 2} error: 0.6000 [0.0415926536 1.2707963268 -0.7415926536]
{'0': 10} error: 0.0000 [0.04159265

In [48]:
prepare_bell_state_circuit(100)

Iteration    0 => Loss 1.7396 (1.0000 0.7396)
Counts: {'00': 100} {'0': 57, '1': 43}
Iteration    2 => Loss 1.6612 (0.9216 0.7396)
Counts: {'00': 98, '11': 2} {'0': 57, '1': 43}
Iteration    3 => Loss 1.5184 (1.0000 0.5184)
Counts: {'00': 100} {'0': 64, '1': 36}
Iteration   12 => Loss 1.3876 (0.8100 0.5776)
Counts: {'00': 95, '11': 5} {'0': 62, '1': 38}
Iteration   16 => Loss 1.3700 (0.4096 0.9604)
Counts: {'00': 82, '11': 18} {'0': 51, '1': 49}
Iteration   23 => Loss 1.1908 (0.5184 0.6724)
Counts: {'00': 86, '11': 14} {'0': 59, '1': 41}
Iteration   77 => Loss 1.0568 (0.6724 0.3844)
Counts: {'00': 91, '11': 9} {'0': 69, '1': 31}
Iteration  273 => Loss 1.0420 (0.3364 0.7056)
Counts: {'00': 21, '11': 79} {'0': 58, '1': 42}
Iteration  404 => Loss 0.9028 (0.5184 0.3844)
Counts: {'00': 86, '11': 14} {'0': 69, '1': 31}
Iteration  490 => Loss 0.8784 (0.3600 0.5184)
Counts: {'00': 80, '11': 20} {'0': 64, '1': 36}
Iteration  494 => Loss 0.6304 (0.2704 0.3600)
Counts: {'00': 76, '11': 24} {'0': 

In [49]:
prepare_bell_state_circuit(1000)

Iteration    0 => Loss 2.0363 (1.0000 1.0363)
Counts: {'00': 1000} {'0': 491, '1': 509}
Iteration    2 => Loss 2.0201 (1.0000 1.0201)
Counts: {'00': 1000} {'0': 495, '1': 505}
Iteration    5 => Loss 1.9920 (1.0000 0.9920)
Counts: {'00': 1000} {'0': 502, '1': 498}
Iteration    9 => Loss 1.9721 (0.9801 0.9920)
Counts: {'00': 995, '11': 5} {'0': 502, '1': 498}
Iteration   10 => Loss 1.8627 (0.9526 0.9101)
Counts: {'00': 988, '11': 12} {'0': 523, '1': 477}
Iteration   11 => Loss 1.7560 (0.8911 0.8649)
Counts: {'00': 972, '11': 28} {'0': 535, '1': 465}
Iteration   42 => Loss 1.7200 (0.9526 0.7674)
Counts: {'00': 988, '11': 12} {'0': 562, '1': 438}
Iteration   46 => Loss 1.7134 (0.9565 0.7569)
Counts: {'00': 989, '11': 11} {'0': 565, '1': 435}
Iteration   48 => Loss 1.6100 (0.9178 0.6922)
Counts: {'00': 979, '11': 21} {'0': 584, '1': 416}
Iteration   55 => Loss 1.5346 (0.9293 0.6053)
Counts: {'00': 982, '11': 18} {'0': 611, '1': 389}
Iteration   66 => Loss 1.3764 (0.8836 0.4928)
Counts: {'00

In [51]:
prepare_bell_state_circuit(1, noisy=True) # this will never converge with only 1 shot

Iteration    0 => Loss 5.0000 (1.0000 4.0000)
Counts: {'00': 1} {'1': 1}
Iteration    2 => Loss 1.0000 (1.0000 0.0000)
Counts: {'00': 1} {'0': 1}

Number of iterations: 999
{'00': 1} error: 1.0000 [-2.1415926536 -1.5707963268 3.1415926536]
{'0': 1} error: 0.0000 [-2.1415926536 -1.5707963268 3.1415926536]
-2.1415926536 -1.5707963268 3.1415926536 (error: 1.0000000000 verify_error: 0.0000000000)


In [52]:
prepare_bell_state_circuit(10, noisy=True)

Iteration    0 => Loss 2.0000 (1.0000 1.0000)
Counts: {'00': 10} {'0': 5, '1': 5}
Iteration    1 => Loss 1.2800 (0.6400 0.6400)
Counts: {'00': 9, '11': 1} {'0': 6, '1': 4}
Iteration   39 => Loss 1.1600 (0.1600 1.0000)
Counts: {'00': 3, '11': 7} {'0': 5, '1': 5}
Iteration   41 => Loss 1.0000 (0.0000 1.0000)
Counts: {'00': 5, '11': 5} {'0': 5, '1': 5}
Iteration   42 => Loss 0.2000 (0.1600 0.0400)
Counts: {'00': 3, '11': 7} {'0': 9, '1': 1}
Iteration   53 => Loss 0.0400 (0.0400 0.0000)
Counts: {'00': 4, '11': 6} {'0': 10}
Iteration  215 => Loss 0.0000 (0.0000 0.0000)
Counts: {'00': 5, '11': 5} {'0': 10}

Number of iterations: 215
{'00': 5, '11': 5} error: 0.0000 [-0.0415926536 1.4707963268 -2.9415926536]
{'0': 10} error: 0.0000 [-0.0415926536 1.4707963268 -2.9415926536]
-0.0415926536 1.4707963268 -2.9415926536 (error: 0.0000000000 verify_error: 0.0000000000)


In [53]:
prepare_bell_state_circuit(100, noisy=True)

Iteration    0 => Loss 1.9216 (1.0000 0.9216)
Counts: {'00': 100} {'0': 52, '1': 48}
Iteration    1 => Loss 1.8464 (1.0000 0.8464)
Counts: {'00': 100} {'0': 54, '1': 46}
Iteration    5 => Loss 1.5776 (1.0000 0.5776)
Counts: {'00': 100} {'0': 62, '1': 38}
Iteration   12 => Loss 1.5080 (0.9604 0.5476)
Counts: {'00': 99, '11': 1} {'0': 63, '1': 37}
Iteration   16 => Loss 1.4116 (0.9216 0.4900)
Counts: {'00': 98, '11': 2} {'0': 65, '1': 35}
Iteration   40 => Loss 1.2308 (0.8464 0.3844)
Counts: {'00': 96, '11': 4} {'0': 69, '1': 31}
Iteration   42 => Loss 1.2296 (0.7396 0.4900)
Counts: {'00': 93, '11': 7} {'0': 65, '1': 35}
Iteration   57 => Loss 1.1348 (0.6724 0.4624)
Counts: {'00': 91, '11': 9} {'0': 66, '1': 34}
Iteration  135 => Loss 0.9872 (0.5776 0.4096)
Counts: {'00': 88, '11': 12} {'0': 68, '1': 32}
Iteration  136 => Loss 0.8720 (0.4624 0.4096)
Counts: {'00': 84, '11': 16} {'0': 68, '1': 32}
Iteration  138 => Loss 0.7988 (0.4624 0.3364)
Counts: {'00': 84, '11': 16} {'0': 71, '1': 29

In [54]:
prepare_bell_state_circuit(1000, noisy=True)

Iteration    0 => Loss 1.9604 (1.0000 0.9604)
Counts: {'00': 1000} {'0': 510, '1': 490}
Iteration    4 => Loss 1.9446 (0.9920 0.9526)
Counts: {'00': 998, '11': 2} {'0': 512, '1': 488}
Iteration    5 => Loss 1.9214 (0.9960 0.9254)
Counts: {'00': 999, '11': 1} {'0': 519, '1': 481}
Iteration    6 => Loss 1.8522 (0.9761 0.8761)
Counts: {'00': 994, '11': 6} {'0': 532, '1': 468}
Iteration    9 => Loss 1.8455 (0.9880 0.8575)
Counts: {'00': 997, '11': 3} {'0': 537, '1': 463}
Iteration   11 => Loss 1.7699 (0.9920 0.7779)
Counts: {'00': 998, '11': 2} {'0': 559, '1': 441}
Iteration   31 => Loss 1.7611 (0.9761 0.7850)
Counts: {'00': 994, '11': 6} {'0': 557, '1': 443}
Iteration   35 => Loss 1.6690 (0.9801 0.6889)
Counts: {'00': 995, '11': 5} {'0': 585, '1': 415}
Iteration   41 => Loss 1.6263 (0.9604 0.6659)
Counts: {'00': 990, '11': 10} {'0': 592, '1': 408}
Iteration   89 => Loss 1.6205 (0.9448 0.6757)
Counts: {'00': 986, '11': 14} {'0': 589, '1': 411}
Iteration   91 => Loss 1.5796 (0.9332 0.6464)


In [55]:
prepare_bell_state_circuit(1000, random_parameter_init=True, noisy=True)

Iteration    0 => Loss 0.3486 (0.2247 0.1239)
Counts: {'00': 737, '11': 263} {'0': 824, '1': 176}
Iteration    1 => Loss 0.3153 (0.1616 0.1537)
Counts: {'00': 701, '11': 299} {'0': 804, '1': 196}
Iteration    5 => Loss 0.3098 (0.1037 0.2061)
Counts: {'00': 661, '11': 339} {'0': 773, '1': 227}
Iteration   18 => Loss 0.2801 (0.0900 0.1901)
Counts: {'00': 650, '11': 350} {'0': 782, '1': 218}

Number of iterations: 999
{'00': 788, '11': 212} error: 0.5760 [2.0521848317 -1.3437963268 -2.6015926536]
{'0': 717, '1': 283} error: 0.5660 [2.0521848317 -1.3437963268 -2.6015926536]
2.0521848317 -1.3437963268 -2.6015926536 (error: 0.5760000000 verify_error: 0.5660000000)
